In [ ]:
# Remove the # below Run this cell if you get a error for the package to download
# pip install python-docx

In [4]:
import csv
import re
import os
from docx import Document
from docx.shared import Pt, Inches, Cm
from datetime import datetime
from collections import defaultdict
from docx.enum.text import WD_BREAK

# Input/output paths
# Change the file name with the sheet/excel (always keep the .csv at the end)
input_file = 'testv2.csv'
output_dir = './orders'
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(name):
    return re.sub(r'[^\w\s-]', '', name).replace(' ', '_')

def get_filename(customer, date_str, ext='.docx'):
    base_name = f"{sanitize_filename(customer)}_{date_str}"
    return os.path.join(output_dir, f"{base_name}{ext}")

def render_invoice(doc, customer, date_str, items):
    header = doc.add_paragraph()
    header.alignment = 1
    run = header.add_run("Lian Sheng Seafood Wholesale")
    run.bold = True
    run.font.size = Pt(14)

    info = doc.add_paragraph("Address: 2105 S Jefferson St, Chicago IL 60616 | Tel: 773-954-1973")
    info.alignment = 1
    for r in info.runs:
        r.font.size = Pt(10)

    meta = doc.add_paragraph()
    meta.alignment = 1
    meta_run = meta.add_run(f"Order Summary: ")
    meta_run.font.size = Pt(10)
    meta_run_bold_cust = meta.add_run(customer)
    meta_run_bold_cust.bold = True
    meta_run_bold_cust.font.size = Pt(10)
    meta.add_run(" | Date: ")
    meta_run_bold_date = meta.add_run(date_str)
    meta_run_bold_date.bold = True
    meta_run_bold_date.font.size = Pt(10)


    table = doc.add_table(rows=1, cols=5)
    table.style = 'Table Grid'
    table.autofit = False
    col_widths = [Inches(1.5), Inches(1), Inches(1.2), Inches(1.6), Inches(1.5)]
    for i, width in enumerate(col_widths):
        table.columns[i].width = width

    headers = ['Description', 'Qty', 'Units (lbs)', 'Price', 'Total Price']
    hdr_cells = table.rows[0].cells
    for i, h in enumerate(headers):
        run = hdr_cells[i].paragraphs[0].add_run(h)
        run.font.size = Pt(10)
        run.bold = True

    total_qty = 0
    total_units = 0
    total_price = 0

    for item in items:
        desc, qty, units, price, total = item
        row = table.add_row().cells
        row[0].text = desc
        row[1].text = str(qty)
        row[2].text = str(units)
        row[3].text = f"${price:.2f}" if isinstance(price, (int, float)) else 'N/A'
        row[4].text = f"${total:.2f}" if isinstance(total, (int, float)) else 'N/A'

        total_qty += qty
        total_units += units
        if isinstance(total, (int, float)):
            total_price += total

    # Add total row
    total_row = table.add_row().cells
    total_row[0].text = "TOTAL"
    total_row[1].text = str(total_qty)
    total_row[2].text = f"{total_units:.2f}"
    total_row[3].text = ""
    total_row[4].text = f"${total_price:.2f}"

    # Add 4 blank rows for manual entries
    for _ in range(3):
        row = table.add_row().cells
        for cell in row:
            cell.text = ""

    # Add an additional empty TOTAL row for manual entries
    manual_total_row = table.add_row().cells
    manual_total_row[0].text = "TOTAL"
    for i in range(1, 5):
        manual_total_row[i].text = ""

    # Now set font size 9 for entire table
    for row in table.rows:
        for cell in row.cells:
            for p in cell.paragraphs:
                for r in p.runs:
                    r.font.size = Pt(9)


# Read CSV and group rows by customer
customers = defaultdict(list)
with open(input_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        customer = row.get('Customer', 'Unknown')
        desc = row.get('Description', '')
        qty = int(row.get('Qty', 0))
        units = int(row.get('Units (lbs)', 0))
        price = float(row.get('Price per pound', 0))
        total = float(row.get('Total Price', 0))
        customers[customer].append([desc, qty, units, price, total])

# Generate one document per customer
now = datetime.now()
date_str = now.strftime("%m.%d.%Y")

for customer, items in customers.items():
    output_docx = get_filename(customer, date_str)
    if os.path.exists(output_docx):
        os.remove(output_docx)

    doc = Document()
    section = doc.sections[0]
    section.top_margin = Cm(1)
    section.bottom_margin = Cm(1)
    section.left_margin = Cm(1)
    section.right_margin = Cm(1)

    render_invoice(doc, customer, date_str, items)
    doc.add_paragraph("\n" + "_" * 135 + "\n")  # separator
    render_invoice(doc, customer, date_str, items)

    doc.save(output_docx)
    print(f"✅ Saved: {output_docx}")

# Combined doc for ease of printing
combined_doc = Document()
section = combined_doc.sections[0]
section.top_margin = Cm(1)
section.bottom_margin = Cm(1)
section.left_margin = Cm(1)
section.right_margin = Cm(1)

first_customer = True
for customer, items in customers.items():
    if not first_customer:
        combined_doc.add_page_break()
    else:
        first_customer = False

    # Render invoice twice on the same page
    render_invoice(combined_doc, customer, date_str, items)
    combined_doc.add_paragraph("\n" + "_" * 135 + "\n") 
    render_invoice(combined_doc, customer, date_str, items)

combined_output_path = os.path.join(output_dir, f"All_Customers_{date_str}.docx")
if os.path.exists(combined_output_path):
    os.remove(combined_output_path)

combined_doc.save(combined_output_path)
print(f"✅ Saved combined document: {combined_output_path}")

✅ Saved: ./orders\Ocean_Fresh_06.28.2025.docx
✅ Saved: ./orders\Blue_Harbor_06.28.2025.docx
✅ Saved: ./orders\88_Market_06.28.2025.docx
✅ Saved: ./orders\Seafood_City_06.28.2025.docx
✅ Saved: ./orders\Seaside_Foods_06.28.2025.docx
✅ Saved combined document: ./orders\All_Customers_06.28.2025.docx
